In [1]:
import numpy as np
import pandas as pd
from equiflow import TableZero

In [2]:
np.random.seed(42)
n = 100
data = {
    'age': np.random.randint(18, 80, size=n),
    'race': np.random.choice(['White', 'Black', 'Asian', 'Hispanic', None],
                             size=n),
    'sex': np.random.choice(['Male', 'Female'], size=n),
    'english': np.random.choice(['Fluent', 'Limited'], size=n),
}

for i in range(1, 11):
    data[f'var{i}'] = np.random.randn(n)

df = pd.DataFrame(data)

In [9]:
data_0 = df.copy()
data_1 = data_0.loc[data_0.sex == "Female"]
data_2 = data_1.loc[data_1.english == "Fluent"]

t = TableZero(dfs = [data_0, data_1, data_2],
              cols = ['race','sex', 'english'],
              normalize=False
              )

t1 = t.table_one()
t1

Cohort        
                          0   1   2
Variable Value                     
race     Black           13   8   3
         Hispanic        22   8   4
         White           25  11   5
         Asian           20  12   3
         Missingness     20  13   3
sex      Female          52  52  18
         Male            48   0   0
         Missingness      0   0   0
english  Limited         58  34   0
         Fluent          42  18  18
         Missingness      0   0   0

In [19]:
from pep440 import is_canonical
is_canonical('0.1.0a1')

True